In [78]:
######################################## Uploading and Merging #############################################################
import pandas as pd
import numpy as np 

filepath = '/Users/justineleaclement/Documents/Projets /Australia_SubProjet/'

# loading the three files needed in the merge 
df_micro = pd.read_csv(filepath +'Microburbs.csv')
df_abs = pd.read_csv(filepath + 'ABS Scrapes-csv full with geo.csv')
df_crime = pd.read_csv(filepath +'crime_rates.csv')

#The three dataframes 
print('head of micro',df_micro.head())
print('head of abs',df_abs.head())
print('head of crime',df_crime.head())

head of micro                                     _google_import_A  \
0  https://nm4.microburbs.com.au/Other-Territorie...   
1  https://nm4.microburbs.com.au/SA/City-of-Playf...   
2    https://nm4.microburbs.com.au/NSW/Sydney/Mascot   
3                                                NaN   
4  https://nm4.microburbs.com.au/Vic/Melbourne/Al...   

                                             __url        suburb_name  \
0                                              NaN  Directions Island   
1                                              NaN     Elizabeth East   
2                                              NaN             Mascot   
3  https://nm4.microburbs.com.au/NSW/Sydney/Mascot         Not found.   
4                                              NaN         Alphington   

                  suburb_city              state  \
0  Regional Other Territories  Other Territories   
1            City of Playford                 SA   
2                      Sydney                NSW   
3 

In [80]:
print(df_crime['Suburb_Full_Name'])

0                                               Not found.
1        Crime rate in Aarons Pass, <a href="https://re...
2        Crime rate in Abbotsbury, <a href="https://red...
3        Crime rate in Abbotsford (NSW), <a href="https...
4        Crime rate in Abercrombie, <a href="https://re...
                               ...                        
13638                                           Not found.
13639                                           Not found.
13640                                           Not found.
13641                                           Not found.
13642                                           Not found.
Name: Suburb_Full_Name, Length: 13643, dtype: object


In [81]:
"""
0                                               Not found.
1        Crime rate in Aarons Pass, <a href="https://re...
2        Crime rate in Abbotsbury, <a href="https://red...
3        Crime rate in Abbotsford (NSW), <a href="https...
4        Crime rate in Abercrombie, <a href="https://re...
                               ...                        
13638                                           Not found.
13639                                           Not found.
13640                                           Not found.
13641                                           Not found.
13642                                           Not found.

Name: Suburb_Full_Name, Length: 13643, dtype: object"""

print("Changing 'Crime rate in Aarons Pass, <a href=https://re...' to just the name of the suburb 'Aarons Pass'")

Changing 'Crime rate in Aarons Pass, <a href=https://re...' to just the name of the suburb 'Aarons Pass'


In [82]:
# Cleaning dataframe
# Remove "Crime rate in " from 'Suburb_Full_Name' column + rename subrub name in df_crime to facilitate merging
df_crime['Suburb_Full_Name'] = df_crime['Suburb_Full_Name'].str.replace('Crime rate in ', '')
df_crime.rename(columns={'Suburb_Full_Name': 'suburb_name'}, inplace=True)
# Extracting only the suburb name before the comma #because before the subrub name was 'name, https...'
df_crime['suburb_name'] = df_crime['suburb_name'].str.split(',', expand=True)[0] #0 keep the first string

# displaying the updated 'crime' DataFrame with only suburb names
print(df_crime['suburb_name'])

0              Not found.
1             Aarons Pass
2              Abbotsbury
3        Abbotsford (NSW)
4             Abercrombie
               ...       
13638          Not found.
13639          Not found.
13640          Not found.
13641          Not found.
13642          Not found.
Name: suburb_name, Length: 13643, dtype: object


In [83]:
#MERGING - micro on abs then crime on merged set - abs first as it is the most complete 

# left join of df_abs with df_micro on 'suburb_name'
merged_df = df_abs.merge(df_micro, on='suburb_name', how='left')
# Perform a left join of the above result with df_crime on 'suburb_name'
merged_df = merged_df.merge(df_crime, on='suburb_name', how='left')

In [84]:
#CHECKING MERGED DATAFRAME

merged_df = merged_df.drop('__url_x', axis=1)
print("Merged LEFT:", merged_df)


#REARRANGING
# reorder columns with 'suburb_name' as the first column for clarity 
first_column = 'suburb_name'
merged_df = merged_df[[first_column] + [col for col in merged_df if col != first_column]]
# drop rows where 'suburb_name' is 'NOT FOUND.' 
# 15 'Not found' # replace not found by nan and remove 
merged_df = merged_df[merged_df['suburb_name'] != 'Not found.']


Merged LEFT:                                          _google_import_B  suburb_name  \
0       https://abs.gov.au/census/find-census-data/qui...    Adaminaby   
1       https://abs.gov.au/census/find-census-data/qui...       Albury   
2       https://abs.gov.au/census/find-census-data/qui...     Alectown   
3       https://abs.gov.au/census/find-census-data/qui...       Alpine   
4       https://abs.gov.au/census/find-census-data/qui...       Argoon   
...                                                   ...          ...   
187664  https://abs.gov.au/census/find-census-data/qui...      Millner   
187665  https://abs.gov.au/census/find-census-data/qui...     Miniyeri   
187666  https://abs.gov.au/census/find-census-data/qui...       Wadeye   
187667  https://abs.gov.au/census/find-census-data/qui...  West Arnhem   
187668  https://abs.gov.au/census/find-census-data/qui...      Hackett   

       suburb_population suburb_median_age suburb_number_of_families  \
0                    339  

In [89]:
unique_suburbs = merged_df['suburb_name'].unique()
print("All unique values present in 'suburb_name':", unique_suburbs)

column_counts = merged_df.count()
print("All non-null values in each columns:", column_counts)

# Number of common 'suburb_name' entries between 'abs' and 'crime'?
common_entries_crime = merged_df['suburb_name'].notnull().sum()
print(f"Common entries between 'abs' and 'crime': {common_entries_crime}")


All unique values present in 'suburb_name': ['Adaminaby' 'Albury' 'Alectown' ... 'Wadeye' 'West Arnhem' 'Hackett']
All non-null values in each columns: suburb_name                                            15319
_google_import_B                                       15319
suburb_population                                      14013
suburb_median_age                                      14013
suburb_number_of_families                              14013
                                                       ...  
chances_of_being_a_victim_of_violent_crime_in_state    13582
chances_of_being_a_violent_victim_in_australia         13582
chances_of_being_a_property_victim_in_suburb           13582
chances_of_being_a_property_victim_in_state            13582
chances_of_being_a_property_victim_in_australia        13582
Length: 251, dtype: int64
Common entries between 'abs' and 'crime': 15319


In [90]:
# Replacing 'Not found.' with np.nan (which represents a missing or null value);
merged_df = merged_df.apply(lambda x: x.map(lambda y: np.nan if y == 'Not found.' else y))

# Reset the index after removing rows °Important°
merged_df.reset_index(drop=True, inplace=True) 

print(df_abs['suburb_financial_median_monthly_mortgage_repayments'])
print(merged_df['suburb_financial_median_monthly_mortgage_repayments'])
print(merged_df['suburb_financial_median_weekly_rent'])


0           $1,083 
1           $1,733 
2           $1,257 
3           $3,033 
4        Not found.
            ...    
15329       $1,733 
15330           $0 
15331           $0 
15332           $0 
15333       $2,500 
Name: suburb_financial_median_monthly_mortgage_repayments, Length: 15334, dtype: object
0        $1,083 
1        $1,733 
2        $1,257 
3        $3,033 
4            NaN
          ...   
15314    $1,733 
15315        $0 
15316        $0 
15317        $0 
15318    $2,500 
Name: suburb_financial_median_monthly_mortgage_repayments, Length: 15319, dtype: object
0        $300 
1        $270 
2        $160 
3        $300 
4          NaN
         ...  
15314    $340 
15315     $70 
15316     $50 
15317     $30 
15318    $374 
Name: suburb_financial_median_weekly_rent, Length: 15319, dtype: object


In [91]:
#'************** Checking numbers - unique - describe - duplicate ******************'
column_counts = merged_df.count()

print("Column counts:")
print()
print(column_counts)

print('abs:',df_abs['suburb_name'].unique())
print('micro:',df_micro['suburb_name'].unique())
print('crime:',df_crime['suburb_name'].unique())
print('merged:',merged_df['suburb_name'].unique()) 

print('abs:',df_abs['suburb_name'].describe()) #15334
print('micro:',df_micro['suburb_name'].describe()) #6344
print('crime:',df_crime['suburb_name'].describe()) #13616
print('merged:',merged_df['suburb_name'].describe()) # 15319 After removing the 15 ‘Not Found.’

print("15319 is the number of suburbs in Australia, crime rate was only available for 13582 suburbs")


Column counts:

suburb_name                                            15319
_google_import_B                                       15319
suburb_population                                      14013
suburb_median_age                                      14013
suburb_number_of_families                              14013
                                                       ...  
chances_of_being_a_victim_of_violent_crime_in_state    13582
chances_of_being_a_violent_victim_in_australia         13582
chances_of_being_a_property_victim_in_suburb           13582
chances_of_being_a_property_victim_in_state            13582
chances_of_being_a_property_victim_in_australia        13582
Length: 251, dtype: int64
abs: ['Adaminaby' 'Albury' 'Alectown' ... 'Wadeye' 'West Arnhem' 'Hackett']
micro: ['Directions Island' 'Elizabeth East' 'Mascot' ... 'Sellicks Beach'
 'Daruka' 'Kadina']
crime: ['Not found.' 'Aarons Pass' 'Abbotsbury' ... 'Whitlam' 'Wright'
 'Yarralumla']
merged: ['Adaminaby' 'Albury' 

In [92]:
print("The following is the head of the merged dataframe = merged_df")

print("Display of five surbubs in the merged df:")
print()
print(merged_df.tail())


# Save the merged DataFrame to CSV and JSON files for safety
merged_df.to_csv('merged_inner_df.csv', index=False)
merged_df.to_json('merged_inner_df.json', orient='records', lines=True)

# Check for duplicates in the entire DataFrame
duplicates = merged_df[merged_df.duplicated()]
print("Number of duplicate rows:", len(duplicates))

# Check for duplicates in the 'suburb_name' column
duplicates = merged_df[merged_df.duplicated(subset=['suburb_name'])]
print("Number of duplicate 'suburb_name' rows:", len(duplicates))

The following is the head of the merged dataframe = merged_df
Display of five surbubs in the merged df:

       suburb_name                                   _google_import_B  \
15314      Millner  https://abs.gov.au/census/find-census-data/qui...   
15315     Miniyeri  https://abs.gov.au/census/find-census-data/qui...   
15316       Wadeye  https://abs.gov.au/census/find-census-data/qui...   
15317  West Arnhem  https://abs.gov.au/census/find-census-data/qui...   
15318      Hackett  https://abs.gov.au/census/find-census-data/qui...   

      suburb_population suburb_median_age suburb_number_of_families  \
15314             2,576                36                       645   
15315               650                22                       121   
15316             1,924                27                       408   
15317               785                27                       173   
15318             3,227                40                       841   

      suburb_number_of_privat